# ZEC example from Section 3.2 

In [1]:
import copy
import numpy as np
import networkx as nx
import xgi
from itertools import permutations
from itertools import combinations
import matplotlib.pyplot as plt

from hyperfunctions import *

In [2]:
G = nx.Graph()
nx.add_path(G,[0,1,2])

## Computing the Z-eigenvector from the original graph

Equation: 

$$\lambda' c_i = \sum_j^N A_{ij} c_j$$

In [3]:
A = nx.to_numpy_array(G)
eigvals, eigvecs = np.linalg.eig(A)

eigvals

array([-1.41421356e+00,  9.77950360e-17,  1.41421356e+00])

In [4]:
lambp = np.sqrt(2)
cent = eigvecs[:,2]
cent

array([0.5       , 0.70710678, 0.5       ])

Now calculate $c_\star,c_\times$ from the obtained centralities.

In [5]:
c_star = np.sqrt(2 * (A[0,1] * cent[0] + A[1,2] * cent[2]) * cent[1]/ (2*lambp)) 
c_star

0.7071067811865475

In [6]:
c_times = np.sqrt(2)*c_star
c_times

1.0

Solutions: Z-eigenvector + normalization condition (norm 1 or norm 2)

In [7]:
bc = np.hstack((cent, [c_star, c_times]))

print(bc) # (0.5, sqrt(2)/2, 0.5, sqrt(2)/2, 1)
print(np.sum(bc)) # 2 + sqrt(2)
print(bc @ bc) # 2.5

[0.5        0.70710678 0.5        0.70710678 1.        ]
3.414213562373095
2.5


In [8]:
# Z1 eigenvector
bc1 = bc / np.sum(bc)
bc1

array([0.14644661, 0.20710678, 0.14644661, 0.20710678, 0.29289322])

In [9]:
# Z2 eigenvector
bc2 = bc / (bc @ bc)
bc2

array([0.2       , 0.28284271, 0.2       , 0.28284271, 0.4       ])

## Check against the tensor

Equation to check, side by side:
$$\lambda c_i = \sum_{jklm}^{N,\star,\times} T_{ijklm} c_j c_k c_l c_m $$


Labels: c_star has index 3, c_times has index 4

In [10]:
# As the hypergraph is uniform, equal weights in the hyperedges do not affect the later outcome
# (they just displace the eigenvalue, but that is something we calculate by hand).
w = 1#/(4*3)

In [11]:
# Manually construct the tensor dictionary
Tdict = {tuple(perm) : w for perm in permutations([0,1,3,4,4])} 
Tdict.update({tuple(perm) : w for perm in permutations([1,2,3,4,4])} )

In [12]:
T = (Tdict, (4,4,4,4,4))
T

({(0, 1, 3, 4, 4): 1,
  (0, 1, 4, 3, 4): 1,
  (0, 1, 4, 4, 3): 1,
  (0, 3, 1, 4, 4): 1,
  (0, 3, 4, 1, 4): 1,
  (0, 3, 4, 4, 1): 1,
  (0, 4, 1, 3, 4): 1,
  (0, 4, 1, 4, 3): 1,
  (0, 4, 3, 1, 4): 1,
  (0, 4, 3, 4, 1): 1,
  (0, 4, 4, 1, 3): 1,
  (0, 4, 4, 3, 1): 1,
  (1, 0, 3, 4, 4): 1,
  (1, 0, 4, 3, 4): 1,
  (1, 0, 4, 4, 3): 1,
  (1, 3, 0, 4, 4): 1,
  (1, 3, 4, 0, 4): 1,
  (1, 3, 4, 4, 0): 1,
  (1, 4, 0, 3, 4): 1,
  (1, 4, 0, 4, 3): 1,
  (1, 4, 3, 0, 4): 1,
  (1, 4, 3, 4, 0): 1,
  (1, 4, 4, 0, 3): 1,
  (1, 4, 4, 3, 0): 1,
  (3, 0, 1, 4, 4): 1,
  (3, 0, 4, 1, 4): 1,
  (3, 0, 4, 4, 1): 1,
  (3, 1, 0, 4, 4): 1,
  (3, 1, 4, 0, 4): 1,
  (3, 1, 4, 4, 0): 1,
  (3, 4, 0, 1, 4): 1,
  (3, 4, 0, 4, 1): 1,
  (3, 4, 1, 0, 4): 1,
  (3, 4, 1, 4, 0): 1,
  (3, 4, 4, 0, 1): 1,
  (3, 4, 4, 1, 0): 1,
  (4, 0, 1, 3, 4): 1,
  (4, 0, 1, 4, 3): 1,
  (4, 0, 3, 1, 4): 1,
  (4, 0, 3, 4, 1): 1,
  (4, 0, 4, 1, 3): 1,
  (4, 0, 4, 3, 1): 1,
  (4, 1, 0, 3, 4): 1,
  (4, 1, 0, 4, 3): 1,
  (4, 1, 3, 0, 4): 1,
  (4, 1, 3

Norm 1 check

In [13]:
# right hand side
apply(T, bc1)

array([0.04415588, 0.06244584, 0.04415588, 0.06244584, 0.08831175])

In [14]:
# Z-eigenvalue
lamb = lambp * (12 * bc1[3] * bc1[4]**2)

# right hand side
lamb * bc1

array([0.04415588, 0.06244584, 0.04415588, 0.06244584, 0.08831175])

Norm 2 check

In [15]:
# right hand side
apply(T, bc2)

array([0.1536   , 0.2172232, 0.1536   , 0.2172232, 0.3072   ])

In [16]:
# Z-eigenvalue
lamb = lambp * (12 * bc2[3] * bc2[4]**2)

# right hand side
lamb * bc2

array([0.1536   , 0.2172232, 0.1536   , 0.2172232, 0.3072   ])